# Introduction

To run this on your machine, make sure your environment is setup correctly (see [installation instructions](README.md))

## Loading libraries

In [ ]:
import numpy as np
from cpmpy import *
import os 

# load puzzles from data
instances_path = os.path.join('data','visual_sudoku','label')
sudoku_puzzles = np.asarray([np.load(os.path.join(instances_path, fname)) for fname in os.listdir(instances_path)])


## A sudoku puzzle

Sudoku is a logic-based number puzzle, played on a partially filled 9x9 grid. The goal is to find the unique solution by filling in the empty grid cells with numbers from 1 to 9 in such a way that each row, each column and each of the nine 3x3 subgrids contain all the numbers from 1 to 9 once and only once.

We now display an example 9x9 puzzle, with some grid cells given and some empty:


In [ ]:
instance = sudoku_puzzles[59]
instance


Note how value 0 represents the empty cells, e.g. the cells whose values are the one we seek.

## Variables and domains

Let's have a look at the problem description again:

- The goal is to find the unique solution by filling in the empty grid cells with numbers from 1 to 9

We will model this with Integer Decision Variables with a value of at least 1 and at most 9, arranged in a matrix just like the given puzzle:

```python
puzzle = intvar(1, 9, shape=instance.shape, name="puzzle")
```


## Modeling the constraints

- each row,
- each column and
- each of the nine 3x3 subgrids contain all the numbers from 1 to 9 once and only once.

We will use the `AllDifferent()` global constraint for this.

```python
model = Model(
    # Constraints on rows and columns
    [AllDifferent(row) for row in puzzle],
    [AllDifferent(col) for col in puzzle.T], # numpy's Transpose
)

# we extend it with the block constraints
# Constraints on blocks
for i in range(0,9, 3):
    for j in range(0,9, 3):
        model += AllDifferent(puzzle[i:i+3, j:j+3]) # python's indexing

# Constraints on values (cells that are not empty)
model += (puzzle[given!=e] == given[given!=e]) # numpy's indexing
```

The last constraint ensures that grid cells that are not empty (e.g. `given != e`) receive their given value.

In [ ]:
# let's put everything into a single function

def get_sudoku_model(given):
    e = 0 # empty cells

    puzzle = intvar(1, 9, shape=instance.shape, name="puzzle")
    model = Model(
        # Constraints on rows and columns
        [AllDifferent(row) for row in puzzle],
        [AllDifferent(col) for col in puzzle.T], # numpy's Transpose
    )

    # we extend it with the block constraints
    # Constraints on blocks
    for i in range(0,9, 3):
        for j in range(0,9, 3):
            model += AllDifferent(puzzle[i:i+3, j:j+3]) # python's indexing

    # Constraints on values (cells that are not empty)
    model += (puzzle[given!=e] == given[given!=e]) # numpy's indexing
    return model, puzzle

## Solving

With the data, variables and constraints set, we can now combine these in a CP model, and use a solver to solve it:

In [ ]:
# Solve and print
model, puzzle = get_sudoku_model(instance)
if model.solve():
    print(puzzle.value())
else:
    print("No solution found")

or display more nicely using some Python libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

# matplotlib/seaborn graphical visualisation
def visu_sudoku(grid, figsize=(6,6)):
    N = int(math.sqrt(grid.shape[0]))

    # block-by-block alternation
    bg = np.zeros(grid.shape)
    for i in range(0,9, 3):
        for j in range(0,9, 3):
            if (i+j) % 2:
                bg[i:i+3, j:j+3] = 1
        
    # the figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    sns.set(font_scale=2)
    sns.heatmap(bg, annot=grid,
                cbar=False, linewidths=1, xticklabels=False, yticklabels=False)
    
    plt.title(f"Sudoku {grid.shape[0]}x{grid.shape[1]}", fontsize=15)
    plt.show()


visu_sudoku(puzzle.value())

## Hyperparameter search 

CPMpy connects with [many solvers](https://cpmpy.readthedocs.io/en/latest/api/solvers.html) through their Python API interface. Those APIs typically expose many parameters that changes the solver's behaviour during search. Carefully tuning these parameters may reduce runtime. This is very welcomed in our setting where we expect to solve a combinatorial problem repeatedly. 

Using OR-Tools CP-SAT solver, the following block shows how to fine-tune some hyperparameters (more available [here](https://cpmpy.readthedocs.io/en/latest/api/solvers/ortools.html#module-cpmpy.solvers.ortools)) on **our sudoku example from above.**

In [ ]:
from cpmpy.tools import ParameterTuner

tunables = {
    "search_branching":[0,1,2],
    "linearization_level":[0,1],
    'symmetry_level': [0,1,2]}
defaults = {
    "search_branching": 0,
    "linearization_level": 1,
    'symmetry_level': 2
}

tuner = ParameterTuner("ortools", model, tunables, defaults)
best_params = tuner.tune(max_tries=100)
best_runtime = tuner.best_runtime
print(f"Fastest in {best_runtime:.4f} seconds, config:", best_params)

print(f'Comparing with default {tuner.base_runtime:.4f} seconds, config: ', defaults)

## Exploratory Data Analysis (part 1)

The dataset used in this tutorial contains 103 Sudoku problems. Having more starting clues generally tends to corrolate with the puzzle's relative level of difficulty. 
To gain further insights into the dataset, we will now visualize the distribution of hardness. 


In [ ]:
# count non-empty cells for each puzzle
counts_non_empty = (sudoku_puzzles.reshape(-1,81) > 0).sum(-1)

# let's plot this 
f, ax = plt.subplots()
sns.histplot(counts_non_empty, binwidth=3, ax=ax)
ax.set_xticks(np.arange(15,85,10))
ax.set_xlabel('number of starting clues')
print()

From the plot, most instances are in the 25-30 range of medium puzzles, then we have fewer rather easy (35+ starting clues) problems. 
The dataset also contains almost 30 fully completed instances. Solving them amount to checking their feasibility. 

In [ ]:
from cpmpy.solvers import CPM_ortools

def solve_instances(array_puzzle, solver_params=dict()):
    runtimes = []
    for given in array_puzzle:
        model, puzzle = get_sudoku_model(given)
        s = CPM_ortools(model)
        # Solve and print
        s.solve(**solver_params)
        runtimes.append(s.status().runtime)
    return runtimes

runtimes = solve_instances(sudoku_puzzles)
print(f'Total runtime: {sum(runtimes):.4f} seconds, avg: {np.mean(runtimes)} s')

**Task: try out different hyperparameters to fine tune on our Sudoku example. Can you improve the total solve runtime on the whole dataset?**